In [1]:
import gurobipy as gp

In [2]:
from gurobipy import GRB
from gurobipy import quicksum

In [3]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-12


In [4]:
Y = {}
for i in range(1,8):
    Y[i] = m.addVar(vtype=GRB.BINARY, name="Y"+str(i))

In [5]:
I = {}
for i in range(1,8):
    I[i] = m.addVar(vtype=GRB.CONTINUOUS, name="I"+str(i))
    
D = {}
for i in range(1,8):
    D[i] = m.addVar(vtype=GRB.CONTINUOUS, name="D"+str(i))
    
X = {}
for i in range(1,8):
    X[i] = m.addVar(vtype=GRB.CONTINUOUS, name="X"+str(i))

In [6]:
P = {1:1, 2:0, 3:1, 4:0, 5:0, 6:0, 7:0, 8:0, 9:1}
QOL_P_parameters = {1:-5, 2:-0.5, 3:-12, 4:-8, 5:-5, 6:-5, 7:-1, 8:-3, 9:-2}
QOL_Y_parameters = {1:-5, 2:-6, 3:-4, 4:-4, 5:-8, 6:-6, 7:-7}
QOL_X_parameters = {1:0.28, 2:0.30, 3:0.25, 4:0.17, 5:0.31, 6:0.246, 7:0.40}

minDosages = {1:20, 2:10, 3:20, 4:10, 5:10, 6:20, 7:20}
maxDosages = {1:80, 2:50, 3:100, 4:100, 5:70, 6:90, 7:50}
baseRegimenDosages = {1:20, 2:0, 3:30, 4:15, 5:0, 6:0, 7:35}
totalDosageCl = 275

Q_threshold = 35

fixedCosts = {1:25, 2:50, 3:10, 4:25, 5:20, 6:30, 7:40}
unitCosts = {1:1, 2:2, 3:1, 4:3, 5:2, 6:1, 7:1}

In [7]:
isDrugInBaseRegimen = {1:True, 2:False, 3:True, 4:True, 5:False, 6:False, 7:True}
m.setObjective( quicksum([(fixedCosts[i] * (1 - Y[i]) if isDrugInBaseRegimen[i] else fixedCosts[i] * Y[i]) for i in range(1,8)])
               + quicksum([unitCosts[i] * (I[i] + D[i]) for i in range(1,8)])
               , GRB.MINIMIZE)

In [8]:
helperXConstraints = {}
minDrugConstraint = {}
maxDrugConstraint = {}
for i in range(1,8):
    helperXConstraints[i] = m.addConstr(X[i] == baseRegimenDosages[i] + I[i] - D[i])
    minDrugConstraint[i] = m.addConstr(X[i] >= minDosages[i] * Y[i])
    maxDrugConstraint[i] = m.addConstr(X[i] <= maxDosages[i] * Y[i])
    
totalDosageConstraint = m.addConstr(quicksum([X[i] for i in range(1,8)]) == totalDosageCl)

QOFConstraint = m.addConstr(quicksum([QOL_P_parameters[i] * P[i] for i in range(1,10)])
                            + quicksum([QOL_Y_parameters[i] * Y[i] for i in range(1,8)])
                            + quicksum([QOL_X_parameters[i] * X[i] for i in range(1,8)])
                            >= Q_threshold
)

In [9]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23 rows, 28 columns and 70 nonzeros
Model fingerprint: 0xcfe4824f
Variable types: 21 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+02]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 3 rows and 10 columns
Presolve time: 0.00s
Presolved: 20 rows, 18 columns, 56 nonzeros
Variable types: 11 continuous, 7 integer (7 binary)
Found heuristic solution: objective 235.0000000

Root relaxation: objective 1.850000e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  185.00000    0    1  235.00000  185.00000  21.3%     -    0s
     0     0  235.00000    0    2  235.00000  235

In [10]:
m.printAttr('X')


    Variable            X 
-------------------------
          Y1            1 
          Y3            1 
          Y4            1 
          Y7            1 
          I1           60 
          I3           70 
          I4           30 
          I7           15 
          X1           80 
          X3          100 
          X4           45 
          X7           50 
